In [90]:
import pandas as pd

In [91]:
df = pd.read_csv("data/Fish.csv")
df.head()

,Species,Weight,Length1,Length2,Length3,Height,Width
0,Bream,242.0,23.2,25.4,30.0,11.5200,4.0200
1,Bream,290.0,24.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,23.9,26.5,31.1,12.3778,4.6961
3,Bream,363.0,26.3,29.0,33.5,12.7300,4.4555
4,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340


In [92]:
perch_df = df[df['Species'] == 'Perch'] #데이터프레임 형태

In [93]:
perch_lhw = perch_df[['Length2', 'Height', 'Width']] #팬시인덱싱
perch_g = perch_df[['Weight']]

In [94]:
from sklearn.model_selection import train_test_split #걍외워

In [95]:
train_input, test_input, train_target, test_target = train_test_split(perch_lhw, perch_g, random_state=42)

In [96]:
train_input.head()

,Length2,Height,Width
80,19.6,5.1376,3.0368
89,22.0,5.8750,3.5250
78,18.7,5.1992,3.1234
76,17.4,4.5880,2.9415
113,36.0,10.6091,6.7408


In [97]:
from sklearn.preprocessing import PolynomialFeatures

In [98]:
poly = PolynomialFeatures(include_bias=False) #인스턴스화
poly.fit([[2, 3]]) #최적화
poly.transform([[2, 3]])

array([[2., 3., 4., 6., 9.]])

In [99]:
poly = PolynomialFeatures(include_bias=False)
poly.fit(train_input)
train_poly = poly.transform(train_input)
train_poly[:5]

array([[  19.6       ,    5.1376    ,    3.0368    ,  384.16      ,
         100.69696   ,   59.52128   ,   26.39493376,   15.60186368,
           9.22215424],
       [  22.        ,    5.875     ,    3.525     ,  484.        ,
         129.25      ,   77.55      ,   34.515625  ,   20.709375  ,
          12.425625  ],
       [  18.7       ,    5.1992    ,    3.1234    ,  349.69      ,
          97.22504   ,   58.40758   ,   27.03168064,   16.23918128,
           9.75562756],
       [  17.4       ,    4.588     ,    2.9415    ,  302.76      ,
          79.8312    ,   51.1821    ,   21.049744  ,   13.495602  ,
           8.65242225],
       [  36.        ,   10.6091    ,    6.7408    , 1296.        ,
         381.9276    ,  242.6688    ,  112.55300281,   71.51382128,
          45.43838464]])

In [100]:
poly.get_feature_names_out()

array(['Length2', 'Height', 'Width', 'Length2^2', 'Length2 Height',
       'Length2 Width', 'Height^2', 'Height Width', 'Width^2'],
      dtype=object)

In [101]:
test_poly = poly.transform(test_input)

In [102]:
from sklearn.linear_model import LinearRegression

In [103]:
lr = LinearRegression()
lr.fit(train_poly, train_target)
lr.score(train_poly, train_target) #학습용

0.9903557670312703

In [104]:
lr.score(test_poly, test_target) #채점용

0.971237620746181

In [105]:
poly = PolynomialFeatures(include_bias=False, degree=5)
poly.fit(train_input)
train_poly = poly.transform(train_input)
test_poly = poly.transform(test_input)
train_poly.shape

(42, 55)

In [106]:
poly.get_feature_names_out()

array(['Length2', 'Height', 'Width', 'Length2^2', 'Length2 Height',
       'Length2 Width', 'Height^2', 'Height Width', 'Width^2',
       'Length2^3', 'Length2^2 Height', 'Length2^2 Width',
       'Length2 Height^2', 'Length2 Height Width', 'Length2 Width^2',
       'Height^3', 'Height^2 Width', 'Height Width^2', 'Width^3',
       'Length2^4', 'Length2^3 Height', 'Length2^3 Width',
       'Length2^2 Height^2', 'Length2^2 Height Width',
       'Length2^2 Width^2', 'Length2 Height^3', 'Length2 Height^2 Width',
       'Length2 Height Width^2', 'Length2 Width^3', 'Height^4',
       'Height^3 Width', 'Height^2 Width^2', 'Height Width^3', 'Width^4',
       'Length2^5', 'Length2^4 Height', 'Length2^4 Width',
       'Length2^3 Height^2', 'Length2^3 Height Width',
       'Length2^3 Width^2', 'Length2^2 Height^3',
       'Length2^2 Height^2 Width', 'Length2^2 Height Width^2',
       'Length2^2 Width^3', 'Length2 Height^4', 'Length2 Height^3 Width',
       'Length2 Height^2 Width^2', 'Length2 Hei

In [107]:
lr.fit(train_poly, train_target)
lr.score(train_poly, train_target) #과대적합

0.9999999999994075

In [108]:
lr.score(test_poly, test_target)

-129.88033371789092

In [109]:
#규제 필요! (스케일드)

In [110]:
from sklearn.preprocessing import StandardScaler

In [111]:
ss = StandardScaler()
ss.fit(train_poly)
ss.transform(train_poly)
train_scaled = ss.transform(train_poly)
test_scaled = ss.transform(test_poly)
#train_scaled[:5]

- 릿지 : 계수를 제곱한 값을 기준으로 규제
    - L2 norm: 모든 계수를 작게 만들지만 0은 되지 않음
- 라쏘 : 계수의 절댓값을 기준으로 규제
    - L1 norm: 불필요한 특성의 가중치를 0으로 만듦

In [112]:
from sklearn.linear_model import Ridge
ridge = Ridge()
ridge.fit(train_scaled, test_scaled)
ridge.score(train_scaled, test_scaled)

ValueError: Found input variables with inconsistent numbers of samples: [42, 14]

In [113]:
import matplotlib.pyplot as plt

In [114]:
train_score = []
test_score = []

In [115]:
alpha_list = [0.001, 0.01, 0.1, 1, 10, 100]
for alpha in alpha_list:
    ridge = Ridge(alpha=alpha)

    ridge.fit(train_sclaed, train_target)

    train_score.append(ridge.score(train_sclaed, train_target))
    test_score.append(ridge.score(test_sclaed, test_target))

NameError: name 'test_sclaed' is not defined

In [88]:
#numpy

In [89]:
ridge = Ridge(alpha=0.1)
ridge.fit(train_sclaed, train_target)

ridge.score(train_sclaed, train_target)
ridge.score(test_scaled, test_target)

0.9826465162736704

In [118]:
lasso = Lasso()
lasso.fit(train_sclaed, train_target)

lasso.score(train_scaled, train_target)

NameError: name 'Lasso' is not defined

In [ ]:
lasso.coef_